In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

In [2]:
def to_datetime_format(value):
    pattern_1 = r'^\d+/\d+/\d+ \d+:\d+$'
    pattern_2 = r'^\d+-\d+-\d+ \d+:\d+$'
    pattern_3 = r'^\d+/\d+/\d+$'
    pattern_4 = r'^\d+-\d+-\d+$'
    
    if re.match(pattern_1, value):
        (date, time) = value.split(' ')
        (day, month, year) = date.split('/')
        (minutes, seconds) = time.split(':')

        year = int(year) if len(year) == 4 else int(year) + 2000
        month = int(month)
        day = int(day)
        minutes = int(minutes)
        seconds = int(seconds)
        
    elif re.match(pattern_2, value):
        (date, time) = value.split(' ')
        (day, month, year) = date.split('-')
        (minutes, seconds) = time.split(':')

        year = int(year) if len(year) == 4 else int(year) + 2000
        month = int(month)
        day = int(day)
        minutes = int(minutes)
        seconds = int(seconds)
        
    elif re.match(pattern_3, value):
        (day, month, year) = value.split('/')

        year = int(year) if len(year) == 4 else int(year) + 2000
        month = int(month)
        day = int(day)
        minutes = 0
        seconds = 0
        
    elif re.match(pattern_4, value):
        (day, month, year) = value.split('-')

        year = int(year) if len(year) == 4 else int(year) + 2000
        month = int(month)
        day = int(day)
        minutes = 0
        seconds = 0
    
    return datetime(year, month, day, minutes, seconds)

def create_numeric_category(col_serie):
    category_temp = pd.factorize(col_serie.str.lower(), sort=True)[0]
    jump = abs(min(category_temp))
    return category_temp + jump

In [3]:
df = pd.read_csv('data/serviciosbomberoshmo.csv', encoding='latin1')
df.head(2)

,FOLIO,TIPO,LUGAR,FECHA,SALIDA,LLEGADA,CAUSA,COLONIA,ESTACION
0,01012016-0007-INZB-2A,INCENDIO DE ZACATE Y BASURA,COMERCIO,1/1/2016,1/1/2016 0:07,1/1/2016 0:16,FUEGO DIRECTO,SIN COLONIA,2.0
1,01012016-0022-INZB-1A,INCENDIO DE ZACATE Y BASURA,CASA,1/1/2016,1/1/2016 0:22,1/1/2016 0:32,FUEGO DIRECTO,LOMAS DE MADRID,1.0


In [4]:
columns = df.columns

print('Faltan datos en:\n')
for col in columns:
    if df[(df[col].isna()) | (df[col].isnull())].size != 0:
        print(col)

Faltan datos en:

TIPO
LUGAR
LLEGADA
CAUSA
COLONIA
ESTACION


In [5]:
# col = 'ESTACION';
# df[(df[col].isna()) | (df[col].isnull())]
# df[col].unique()
# df[df[col] == 'ENDA']
# df[col].replace('-','',inplace=True)

df['TIPO'].fillna('OTROS', inplace=True)

df['LUGAR'].replace('-','OTROS',inplace=True)
df['LUGAR'].replace('ENDA','OTROS',inplace=True)
df['LUGAR'].fillna('OTROS', inplace=True)

df['CAUSA'].fillna('OTROS', inplace=True)

df['COLONIA'].fillna('SIN COLONIA', inplace=True)

df['ESTACION'].fillna(0, inplace=True)


In [6]:
df_salida_llegada = df[df['LLEGADA'].isna()!=True][['SALIDA','LLEGADA']].copy()

salida = df_salida_llegada['SALIDA'].apply(to_datetime_format)
llegada = df_salida_llegada['LLEGADA'].apply(to_datetime_format)

timepo_de_llegada = llegada - salida

timepo_de_llegada.median()

Timedelta('0 days 00:08:00')

In [7]:
llegada_nueva = df[df['LLEGADA'].isna()]['SALIDA'].apply(to_datetime_format) + timepo_de_llegada.median()

df.loc[df['LLEGADA'].isna(),'LLEGADA'] = llegada_nueva.apply(lambda x: x.strftime('%d-%m-%Y %H:%M'))

In [8]:
temp = df['SALIDA'].apply(to_datetime_format).apply(lambda x: x.day)
temp.head(2)

0    1
1    1
Name: SALIDA, dtype: int64

In [9]:
df['TIPO_ID'] = create_numeric_category(df['TIPO'])
df['LUGAR_ID'] = create_numeric_category(df['LUGAR'])
df['CAUSA_ID'] = create_numeric_category(df['CAUSA'])
df['COLONIA_ID'] = create_numeric_category(df['COLONIA'])

In [10]:
df['SALIDA_AÑO'] = df['SALIDA'].apply(to_datetime_format).apply(lambda x: x.year)
df['SALIDA_MES'] = df['SALIDA'].apply(to_datetime_format).apply(lambda x: x.month)
df['SALIDA_DIA'] = df['SALIDA'].apply(to_datetime_format).apply(lambda x: x.day)

df['LLEGADA_AÑO'] = df['LLEGADA'].apply(to_datetime_format).apply(lambda x: x.year)
df['LLEGADA_MES'] = df['LLEGADA'].apply(to_datetime_format).apply(lambda x: x.month)
df['LLEGADA_DIA'] = df['LLEGADA'].apply(to_datetime_format).apply(lambda x: x.day)

In [11]:
df['FECHA_AÑO'] = df['FECHA'].apply(to_datetime_format).apply(lambda x: x.year)
df['FECHA_MES'] = df['FECHA'].apply(to_datetime_format).apply(lambda x: x.month)
df['FECHA_DIA'] = df['FECHA'].apply(to_datetime_format).apply(lambda x: x.day)
df['FECHA_DIA_SEMANA'] = df['FECHA'].apply(to_datetime_format).apply(lambda x: x.weekday())

In [16]:
salida = df['SALIDA'].apply(to_datetime_format)
llegada = df['LLEGADA'].apply(to_datetime_format)

tiempo_de_llegada = llegada - salida

df['TIEMPO_DE_LLEGADA_MINUTOS'] = tiempo_de_llegada.apply(lambda x: x.seconds // 60)

In [13]:
df.to_csv('data/serviciosbomberoshmo_ready.csv', encoding='latin1',index = None)